# LangChain Expression Language (LCEL)
LangChain Expression Language (LCEL) is the recommended approach to building chains in LangChain, having superseded the traditional methods (including LLMChain). LCEL gives us a more flexible system for building chains. The pipe operator | is used by LCEL to chain together components. Let's see how we'd construct an LLMChain using LCEL.


In [ ]:
#Traditional chains 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate



In [ ]:
prompt_template = "Give me a small report on {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)



 # input | prompt_template | llm | output

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0,
)



In [ ]:
from langchain_core.output_parsers import StrOutputParser

#from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()



In [ ]:
from langchain_classic.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt , output_parser=output_parser)

result = chain.invoke("langchain expression language")
print(result)   

In [ ]:
#Through the LLMChain class, we can place each of our components in a linear chain.

from langchain_classic.chains import LLMChain
#from langchain.chains import LLMChain

lcel_chain = prompt | llm | output_parser

result = lcel_chain.invoke("langchain expression language")

print(result)

In [ ]:
from IPython.display import display, Markdown

display(Markdown(result))

In [ ]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other.invoke(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)


In [ ]:
def add_five(x):
    return x+5   # 3+5 =8

def sub_two(x):
    return x-2   # 3-2  or 8 -2

def mul_seven(x):
    return x*7

In [ ]:
add_five_runnable = Runnable(add_five)
sub_five_runnable = Runnable(sub_two)
mul_five_runnable = Runnable(mul_seven)

In [ ]:
chain = (add_five_runnable).__or__(sub_five_runnable).__or__(mul_five_runnable)

chain.invoke(4)

In [ ]:
(3).__add__(5)  # 8

In [ ]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

chain.invoke(4)

# LCEL RunnableLambda
The RunnableLambda class is LangChain's built-in method for constructing a runnable object from a function. It does the same thing as the custom Runnable class we created earlier. Let's try it out with the same functions.

In [ ]:
def add_five(x):
    return x+5   # 3+5 =8

def sub_two(x):
    return x-2   # 3-2  or 8 -2

def mul_seven(x):
    return x*7

In [ ]:
from langchain_core.runnables import RunnableLambda

add_five_runnable = RunnableLambda(add_five)
sub_five_runnable = RunnableLambda(sub_two)
mul_five_runnable = RunnableLambda(mul_seven)

chain = add_five_runnable | sub_five_runnable | mul_five_runnable 
chain.invoke(4)

In [ ]:
# lets try something complex
prompt_str = "give me a small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

chain = prompt | llm | output_parser

In [ ]:
result = chain.invoke("AI")
display(Markdown(result))

### Here, we define two functions: extract_fact will remove the first paragraph (typically the introduction), and replace_word will replace the substring "RAG" with "hot potato"

In [ ]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x
    
old_word = "RAG"
new_word = "hot potato"

def replace_word(x):
    return x.replace(old_word, new_word)


In [ ]:
extract_fact_runnable = RunnableLambda(extract_fact)
replace_word_runnable = RunnableLambda(replace_word)

chain = prompt | llm | output_parser | extract_fact_runnable | replace_word_runnable
result = chain.invoke("retrieval augmented generation")


In [ ]:
from IPython.display import display, Markdown
display(Markdown(result))

# LCEL RunnableParallel and RunnablePassthrough
LCEL provides us with various Runnable classes that allow us to control the flow of data and execution order through our chains. Two of these are RunnableParallel and RunnablePassthrough.

RunnableParallel allows us to run multiple Runnable instances in parallel, acting almost like a Y-fork in the chain.

RunnablePassthrough — allows us to pass through a variable to the next Runnable without modification.

To see these runnable in action, we will create two data sources. Each source provides specific information, but to answer the question, we will need both to be fed to the LLM.

In [ ]:
#!pip install langchain-community
#!pip install docarray

In [ ]:
#
try:
    from langchain_openai import OpenAIEmbeddings
    from langchain_community.vectorstores import DocArrayInMemorySearch



    embedding = OpenAIEmbeddings()
    vecstore_a = DocArrayInMemorySearch.from_texts(
        ["half the info is here", "James' birthday is December the 7th"],
        embedding=embedding,
    )



    vecstore_b = DocArrayInMemorySearch.from_texts(
        ["the other half of the info is here", "James was born in 1994"],
        embedding=embedding,
    )

    
except ImportError as e:
    print("Missing package required for DocArrayInMemorySearch:", e)
    print("Install with: python -m pip install docarray (activate your venv first).")


In [ ]:
prompt_str = """Using the context provided, answer the user's question.
Context: 
{context_a}
{context_b}

Question:
{question}

Answer: """


In [ ]:
from langchain_core.prompts import (
 ChatPromptTemplate,
 SystemMessagePromptTemplate,
 HumanMessagePromptTemplate,
)

prompt = ChatPromptTemplate.from_messages([
 SystemMessagePromptTemplate.from_template(prompt_str),
 HumanMessagePromptTemplate.from_template("{question}") 
])


In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)


In [ ]:
chain = retrieval | prompt | llm | output_parser
result = chain.invoke("What was the date when James was born")
result


In [ ]:
prompt